# Clean & Preprocessing 

## 1. Imports and Load Data

In [1]:
import pandas as pd
import re
import os

In [ ]:
DATA_PATH = "../data/raw/all_california_gym_reviews.csv"
if not os.path.exists(DATA_PATH):
    print(f"Error: File not found at {DATA_PATH}")
else:
    # Load the data
    df = pd.read_csv(DATA_PATH, encoding='utf-8-sig')
    print(f"Successfully loaded {len(df):,} reviews")

Successfully loaded 658 reviews


## 2. Drop empty comments

In [ ]:
df = df.dropna(subset=['comment'])
df = df[df['comment'].str.strip() != '']

## 3. Créer les labels de sentiment

In [7]:
def rating_to_sentiment(rating):
    if rating >= 4:
        return 'positif'
    elif rating == 3:
        return 'neutre'
    else:
        return 'negatif'

df['sentiment'] = df['rating'].apply(rating_to_sentiment)

# Vérifier la distribution
print(df['sentiment'].value_counts(normalize=True))

sentiment
positif    0.691882
negatif    0.217712
neutre     0.090406
Name: proportion, dtype: float64


## 4. Nettoyage du texte

In [ ]:
def clean_text_light(text):
    """
    Nettoyage léger : conserve la ponctuation importante pour le sentiment.
    """
    
    # Supprimer les URLs, mentions, hashtags 
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\@\w+|\#\w+', '', text)
    
    
    # Remplacer les séquences d'espaces par un seul espace
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

df['comment_clean'] = df['comment'].apply(clean_text_light)

# Exemples
print("Exemples de commentaires nettoyés :")
for i in range(3):
    print(f"  Original : {df.loc[i, 'comment']}")
    print(f"  Nettoyé : {df.loc[i, 'comment_clean']}\n")

Exemples de commentaires nettoyés :
  Original : Bonjour/Bonsoir Est ce que l'administration/la direction est au courant de la condition insalubre et l'état détérioré et dégoutant des douches des vestiaires pour hommes? Si Non, c'est très grave Si Oui, c'est encore pire Je suis pas un expert, mais un petit coup de maintenance ne peut pas être aussi compliqué. 1. Démonter les séparations en bois (4 heures/worker) 2. Un bon coup de nettoyage en Kärcher en appliquant des produits de désinfection et nettoyant Mir et par-terre (allez 8 heures max/worker) 3. Démonter les douches et les remplacer par des nouvelles (4 heures max/worker) 4. Monter la nouvelle séparation. Peut-être pas en bois cette fois-ci pour éviter les moisissures ? ;) (8 heures max/worker) En total, pour un seul vestiaire, ça coûtera 24 heures par travailleur. En embauchant 2 travailleurs cela nous mène à 12 heures chaque, 3 travailleurs vous coûtera 8 heures chaque. Quant au financement, je pense pas que c'est la mer à boi

## 5. Supprimer les colonnes inutiles

In [9]:
df = df.drop(columns=['id', 'name', 'source', 'location', 'date', 'comment'])

## 6. Encodage numérique des labels (pour compatibilité)

In [10]:
sentiment_map = {'negatif': 0, 'neutre': 1, 'positif': 2}
df['sentiment_encoded'] = df['sentiment'].map(sentiment_map)

## 7. Sauvegarder dataset general nettoyé & préparé  

In [11]:
df.to_csv("../data/cleaned/cleaned_reviews_general.csv", index=False)